In [1]:
import numpy as np
import tensorflow as tf
import os
import sys
from sklearn.preprocessing import StandardScaler 

C:\Users\victor_accete\Anaconda3\envs\curatornet\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\victor_accete\Anaconda3\envs\curatornet\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\victor_accete\Anaconda3\envs\curatornet\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\victor_accete\Anaconda3\e

In [2]:
module_path = os.path.abspath(os.path.join('../src'))
if module_path not in sys.path:
    sys.path.append(module_path)
    
from utils import load_embeddings_and_ids, concatenate_featmats
from Networks import CuratorNet_Precomputation

In [3]:
resnet50 = load_embeddings_and_ids('../data/resnet_embeddings.npy')

In [4]:
embedding_list = [
    resnet50,
]

In [5]:
artwork_ids_set = set()
for embedding in embedding_list:
    if len(artwork_ids_set) == 0:        
        artwork_ids_set.update(embedding['index2id'].values())
    else:
        artwork_ids_set.intersection_update(embedding['index2id'])
artwork_ids = list(artwork_ids_set)
artwork_id2index = {_id:i for i,_id in enumerate(artwork_ids)}
n_artworks = len(artwork_ids)
n_artworks

13297

In [6]:
featmat_list = [tmp['featmat'] for tmp in embedding_list]
id2index_list = [tmp['id2index'] for tmp in embedding_list]
concat_featmat = concatenate_featmats(artwork_ids, featmat_list, id2index_list)

In [7]:
concat_featmat = StandardScaler().fit_transform(concat_featmat)

In [8]:
concat_featmat.shape

(13297, 2048)

In [9]:
!dir C:\Users\victor_accete\CuratorNet\experiments\curatornet_10m

 O volume na unidade C nÆo tem nome.
 O N£mero de S‚rie do Volume ‚ 789E-3B0A

 Pasta de C:\Users\victor_accete\CuratorNet\experiments\curatornet_10m

13/12/2021  23:12    <DIR>          .
13/12/2021  23:12    <DIR>          ..
13/12/2021  23:12         8.649.608 .data-00000-of-00001
13/12/2021  23:12             1.204 .index
13/12/2021  23:12           193.468 .meta
13/12/2021  23:12               191 checkpoint
13/12/2021  23:17             1.382 train_logs.csv
               5 arquivo(s)      8.845.853 bytes
               2 pasta(s)   287.453.687.808 bytes dispon¡veis


In [10]:
MODEL_PATH = "C:\\Users\\victor_accete\\CuratorNet\\experiments\\curatornet_10m\\"
MODEL_PATH

'C:\\Users\\victor_accete\\CuratorNet\\experiments\\curatornet_10m\\'

In [11]:
print(tf.train.latest_checkpoint(MODEL_PATH))

C:\Users\victor_accete\CuratorNet\experiments\curatornet_10m\


In [12]:
with tf.Graph().as_default():
    network = CuratorNet_Precomputation(
        pretrained_embedding_dim=concat_featmat.shape[1],
        item_layer_units=[200,200],
    )
    gpu_options = tf.GPUOptions(
            per_process_gpu_memory_fraction=0.99,
            allow_growth=True
        )
    config = tf.ConfigProto(gpu_options=gpu_options)
    with tf.Session(config=config) as sess:
        saver = tf.train.Saver()
        saver.restore(sess, tf.train.latest_checkpoint(MODEL_PATH))
        item_vectors = network.precompute_tensors(sess, concat_featmat)




Instructions for updating:
Use keras.layers.dense instead.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Use standard file APIs to check for files with this prefix.
INFO:tensorflow:Restoring parameters from C:\Users\victor_accete\CuratorNet\experiments\curatornet_10m\


In [13]:
item_vectors.shape

(13297, 200)

In [14]:
item_vectors.dump(MODEL_PATH + "item_vectors.npy")
with open(MODEL_PATH + 'ids', 'w') as f:
    for _id in artwork_ids:
        f.write('%s\n' % _id)